# Select segmentation parameters

To segment cells, we use cellpose. This requires a few parameters to be set. 
We will use the following parameters:

- `ops["cellpose_flow_threshold"]`
- `ops["cellpose_rescale"]`
- `ops["cellpose_model"]`
- `ops["segmentation_channels"]`


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
data_path = "becalia_rabies_barseq/BRAC8501.6a/chamber_07"
prefix = "genes_round_4_1"
ref_tile_index = 0  # which of the reference tiles do we want to use for plots
use_gpu = False  # whether to use the GPU for the analysis
window_size = 1000  # size of the window for the analysis, None for the whole image

In [ ]:
import iss_preprocess as iss
import numpy as np

ops = iss.io.load.load_ops(data_path)
tile_coors = ops["genes_ref_tiles"][ref_tile_index]
print(tile_coors)

In [ ]:
# Loading data
print(f'Using channel(s) {ops["segmentation_channels"]}')
stack, bad_pixels = iss.pipeline.load_and_register_tile(
    data_path, tile_coors, prefix, filter_r=False
)
if stack.ndim == 4:
    stack = stack[..., 0]  # The is only one round
stack = stack[..., ops["segmentation_channels"]]
stack = np.nanmean(stack, axis=-1)
if window_size is not None:
    center = [x // 2 for x in stack.shape]
    stack = stack[
        center[0] - window_size // 2 : center[0] + window_size // 2,
        center[1] - window_size // 2 : center[1] + window_size // 2,
    ]

Change the list of flow and rescale factors to try. Without GPU, cellpose is slow and
it might be better to try only a few parameters.

In [ ]:
# Doing segmentation
flow_threshold = [0.35, 0.4, 0.65]  # threshold
rescale_factor = [0.4]  # rescale
for flow in flow_threshold:
    ops["cellpose_flow_threshold"] = flow
    for rescale in rescale_factor:
        ops["cellpose_rescale"] = rescale
        masks = iss.pipeline.segment.cellpose_segmentation(
            stack,
            channels=(0, 0),
            flow_threshold=ops["cellpose_flow_threshold"],
            min_pix=0,
            dilate_pix=0,
            rescale=ops["cellpose_rescale"],
            model_type=ops["cellpose_model"],
            use_gpu=use_gpu,
        )
        fig = iss.pipeline.diagnostics.check_segmentation(
            data_path,
            tile_coors[0],
            prefix,
            reference="genes_round_4_1",
            stitched_stack=stack,
            masks=masks,
            save_fig=False,
        )
        plot_masks = masks.copy().astype(float)
        plot_masks[plot_masks == 0] = np.nan
        fig.axes[0].imshow(plot_masks[::2, ::2], alpha=0.2, cmap="prism")
        fig.axes[0].set_title(
            f"Segmentation params: flow = {ops['cellpose_flow_threshold']},"
            + f" rescale = {ops['cellpose_rescale']}, {ops['cellpose_model']}"
        )
        fig.set_size_inches(5, 5)